# Davies Meyer
Debemos retornar una función de hash que sea una función de compresión a partir de la contrucción de Davies-Meyer.

Por ende, debemos obtener una llave y valor a partir del input, encryptar el valor con la llave, y posteriormente hacer un XOR entre la llave y el valor encriptado. Algo a notar, es que la construcción no requiere necesariamente que la llave y el mensaje sean del mismo tamaño, pero para el XOR se necesita tener que ambos sean del mismo tamaño. Por ende, se trunca al valor más pequeño entre ambos (en python eso se hace por default al usar `zip`).

In [2]:
def davies_meyer(encrypt, l_key: int, l_message: int):
    def hash(key_message):
        key = key_message[:l_key] # obtenemos la llave
        message = key_message[l_key:l_key + l_message] # obtenemos el valor
        
        encrypted_message = encrypt(key, message) # encriptamos
        hashed_message = bytearray(a ^ b for (a, b) in zip(encrypted_message, key)) # hacemos el XOR
        return hashed_message
    return hash

In [7]:
# TEST
#def identity(key: bytearray, value: bytearray) -> bytearray:
#    return value

#comprension = davies_meyer(identity, 16, 16)
#key = bytearray(b'aca hay un mensa')
#text = bytearray(b'y aca hay otro m')

#value = comprension(key + text)
#value

# Padding
Debemos aplicar padding. Como visto en clases, se debe rellenar el último bloque con un 1 seguido de los suficientes 0 como para llegar a terminar el último bloque del mismo tamaño del resto. Posteriormente, se agrega un bloque final que indica el tamaño del mensaje original (módulo 2^(largo del bloque)). Algo a destacar, es que este bloque final podría estar en big o little endian. Arbitrariamente decidí que esté en big endian, dado que no estaba especificado en el enunciado.

In [13]:
import math

def pad(message: bytearray, l_block: int):
    message_length = len(message)
    n_blocks = math.ceil(message_length / l_block) # obtenemos cantidad de bloques originales
    missing_bytes = n_blocks * l_block - message_length # obtenemos cuántos bytes faltan agregar
    
    added_bytes = bytearray()
    if missing_bytes:
        added_bytes += bytearray(b'1') # primero un 1
        added_bytes += bytearray(b'0'*(missing_bytes - 1)) # rellenamos con 0
    
    coded_length = message_length % (2 ** l_block) # de acuerdo a las diapositivas de clases
    last_block = bytearray(coded_length.to_bytes(l_block,'big')) # big endian elegido arbitrariamente
    padded_message = message + added_bytes + last_block

    return padded_message
    

In [8]:
# TEST
#pad(bytearray(b'asdfasdfdsafsaasdfa'), 16)

# Merke Damgard
A partir de un vector de inicialización, dividimos el mensaje (con padding) en bloques, vamos bloque por bloque generando h_i. Se retorna el h final.

In [10]:
def merkle_damgard(IV: bytearray, comp, l_block: int):
    def hash(message):
        padded_message = pad(message, l_block)
        h = IV # inicialmente empieza el vecotr de inicialización
        for i in range(0, len(padded_message), l_block):
            block = padded_message[i: i + l_block] # obtenemos el siguiente bloque
            h = comp(h + block) # calculamos el siguiente h
        return h # retornamos el h final
    return hash


In [16]:
# TEST
# comprension = davies_meyer(identity, 16, 16)
# hash = merkle_damgard(bytearray(b'0123456789012345'), comprension, 16)
# h1 = hash(bytearray(b'serasefasfasdfasdfasdfassdfasd'))
# h1